## Spotify Analysis - Danceability

In [ ]:
with open('./share/common.py') as fin:
   exec(fin.read())

with open('./share/matplotlibconf.py') as fin:
    exec(fin.read())


In [ ]:
df = pd.read_csv('./data/SpotifyFeatures.csv')

In [ ]:
df.loc[df['genre'] == "Children’s Music", 'genre'] = "Children's Music"

In [ ]:
df['genre'].unique()

In [ ]:
# df = pd.get_dummies(df, columns=['genre'], prefix=['genre'])

In [ ]:
df.columns

In [ ]:
df.loc[(df['popularity'] < 34), 'popularity' ] = 0
df.loc[(df['popularity'] >= 34) & (df['popularity'] < 52 ), 'popularity' ] = 1
df.loc[(df['popularity'] >= 52) , 'popularity' ] = 2

## The same feature as ANN-popularity part

features = ['popularity',
       'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', "genre_A Capella", 'genre_Alternative', 'genre_Anime', 'genre_Blues',
       "genre_Children's Music", 'genre_Classical', 'genre_Comedy',
       'genre_Country', 'genre_Dance', 'genre_Electronic', 'genre_Folk',
       'genre_Hip-Hop', 'genre_Indie', 'genre_Jazz', 'genre_Movie',
       'genre_Opera', 'genre_Pop', 'genre_R&B', 'genre_Rap', 'genre_Reggae',
       'genre_Reggaeton', 'genre_Rock', 'genre_Ska', 'genre_Soul',
       'genre_Soundtrack', "genre_World"]

In [ ]:
features = ['popularity',
       'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness']

In [ ]:
df = df[features]
df.head()

In [ ]:
import joblib
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import scikitplot as skplt

In [ ]:
X = df.drop('popularity', axis=1)
y = df['popularity']
feature_names = X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

In [ ]:
X_train.head()

In [ ]:
y_train.describe()

In [ ]:
sns.distplot(y_train)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
clf = RandomForestClassifier(n_estimators=600)
clf.fit(X_train_scaled, y_train)


In [ ]:
clf.get_params()


In [ ]:
print(f"Training Data Score: {clf.score(X_train_scaled, y_train):.2f}")
print(f"Testing Data Score: {clf.score(X_test_scaled, y_test):.2f}")

In [ ]:
# param_grid =   { 
#     'bootstrap': [True],
#     'max_depth': [40, 80, 120],
#     'n_estimators': [100, 300, 600]}
# rfc_grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, verbose=1 )

In [ ]:
param_grid =   { 
    'bootstrap': [True],
    'max_depth': [40, 80, 120],
    'n_estimators': [100, 300, 600]}
rfc_grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, verbose=1 )

In [ ]:
rfc_grid.fit(X_train_scaled, y_train)

In [ ]:
rfc_grid

In [ ]:
print(rfc_grid.best_params_)
print(rfc_grid.best_score_)

In [ ]:
print(f"Training Data Score: {rfc_grid.score(X_train_scaled, y_train):.2f}")
print(f"Testing Data Score: {rfc_grid.score(X_test_scaled, y_test):.2f}")


In [ ]:
rfc_grid_prediction = rfc_grid.predict(X_test_scaled)

In [ ]:
cf_matrix = confusion_matrix(y_test, rfc_grid_prediction)
print(classification_report(y_test, rfc_grid_prediction))

In [ ]:
skplt.metrics.plot_confusion_matrix(y_test, rfc_grid_prediction, figsize=(5,5))


In [ ]:
# save model
model_filename = 'models/rfc_pop_model.h5'
joblib.dump(rfc_grid, model_filename)

In [ ]:
# save scaler
scaler_filename = 'models/rfc_pop_scaler.pkl'
joblib.dump(scaler, scaler_filename)

In [ ]:
# test loading model and scaler
model = joblib.load(model_filename)
scaler = joblib.load(scaler_filename)


In [ ]:
X_test_scaled_test = scaler.transform(X_test)

y_test_hat = model.predict(X_test_scaled_test)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_test_hat)
print('Test accuracy', acc)